In [13]:
import numpy as np
import matplotlib.pyplot as plt
from camutils import calibratePose, Camera, triangulate
import pickle
import visutils
from calibrate import create_calibration_pickle
from decoder import decode
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from skimage import transform
import cv2

In [16]:
def reconstruct(imprefixL,imprefixR,threshold,camL,camR):
    """
    Performing matching and triangulation of points on the surface using structured
    illumination. This function decodes the binary graycode patterns, matches 
    pixels with corresponding codes, and triangulates the result.
    
    The returned arrays include 2D and 3D coordinates of only those pixels which
    were triangulated where pts3[:,i] is the 3D coordinte produced by triangulating
    pts2L[:,i] and pts2R[:,i]

    Parameters
    ----------
    imprefixL, imprefixR : str
        Image prefixes for the coded images from the left and right camera
        
    threshold : float
        Threshold to determine if a bit is decodeable
   
    camL,camR : Camera
        Calibration info for the left and right cameras
        
    Returns
    -------
    pts2L,pts2R : 2D numpy.array (dtype=float)
        The 2D pixel coordinates of the matched pixels in the left and right
        image stored in arrays of shape 2xN
        
    pts3 : 2D numpy.array (dtype=float)
        Triangulated 3D coordinates stored in an array of shape 3xN
    
    color_mask : 3D numpy.array (dtype=float)
        The RGB color of each pixels, stored in the arrays of 3xN
        
    """

    # Decode the H and V coordinates for the two views
    HL, HLmask = decode(imprefixL, 0, threshold)
    HR, HRmask = decode(imprefixR, 0, threshold)
    VL, VLmask = decode(imprefixL, 20, threshold)
    VR, VRmask = decode(imprefixR, 20, threshold)
    
    # Read foreground and background for the scanned objects
    leftB = plt.imread(prefix+ "/color_C0_00.png")
    leftF = plt.imread(prefix + "/color_C1_01.png")
    rightB = plt.imread(prefix + "/color_C0_00.png")
    rightF = plt.imread(prefix + "/color_C1_01.png")
    
    # Threshold the foreground and background
    objMaskL = np.array(np.abs(leftF-leftB)>threshold)
    objMaskR = np.array(np.abs(rightF-rightB)>threshold)

    # Construct the combined 20 bit code C = H + 1024*V and mask for each view
    CL = (HL + 1024 * VL) * HLmask * VLmask * objMaskL
    CR = (HR + 1024 * VR) * HRmask * VRmask * objMaskR
    
    # Find the indices of pixels in the left and right code image that 
    # have matching codes. If there are multiple matches, just
    # choose one arbitrarily.
    Lmatching = np.intersect1d(CL, CR, return_indices=True)[1]
    Rmatching = np.intersect1d(CR, CL, return_indices=True)[1]
    
    # Let CL and CR be the flattened arrays of codes for the left and right view
    # Suppose you have computed arrays of indices matchL and matchR so that 
    # CL[matchL[i]] == CR[matchR[i]] for all i.  The code below gives one approach
    # to generating the corresponding pixel coordinates for the matched pixels.
    CL = CL.flatten()
    CR = CR.flatten()
    h = HL.shape[0]
    w = HL.shape[1]
    
    xx,yy = np.meshgrid(range(w),range(h))
    xx = np.reshape(xx,(-1,1))
    yy = np.reshape(yy,(-1,1))
    pts2R = np.concatenate((xx[Rmatching].T,yy[Rmatching].T),axis=0)
    pts2L = np.concatenate((xx[Lmatching].T,yy[Lmatching].T),axis=0)

    # Now triangulate the points
    pts3=triangulate(pts2L,camL,pts2R,camR)
    
    #Record the color from one of the images
    color_mask = leftF[pts2L[1,:], pts2L[0,:],:].T
    assert color_mask.shape = np.ones((h,w))
    
    return pts2L,pts2R,pts3D,color_mask

In [11]:
# create calibration.pickle file and get intrinsic parameters for camera
create_calibration_pickle()

Estimated camera intrinsic parameter matrix K
[[1.40530791e+03 0.00000000e+00 9.62167364e+02]
 [0.00000000e+00 1.40389402e+03 5.90915958e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Estimated radial distortion coefficients
[[-5.23054050e-03  8.05964926e-02  2.01744099e-05 -3.90736081e-03
  -1.08096643e-01]]
Individual intrinsic parameters
fx =  1405.3079095113371
fy =  1403.8940206811462
cx =  962.167364213723
cy =  590.9159576570755


In [12]:
# load in the intrinsic camera parameters from 'calibration.pickle'
params = np.load("calibration.pickle",allow_pickle=True)

'''
cam.f : float --- camera focal length (in units of pixels)
cam.c : 2x1 vector --- offset of principle point
cam.R : 3x3 matrix --- camera rotation
cam.t : 3x1 vector --- camera translation
'''
f = (params["fx"] + params["fy"])/2
c = np.array([params["cx"], params["cy"]]).reshape(2, 1)
R = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(3, 3)
t = np.array([0, 0, 0]).reshape(3, 1)

camL = Camera(f, c, R, t)
camR = Camera(f, c, R, t)

# load in the left and right images and find the coordinates of
# the chessboard corners using OpenCV
imgL = plt.imread('calib_jpg_u/frame_C0_01.jpg')
ret, cornersL = cv2.findChessboardCorners(imgL, (8,6), None)
pts2L = cornersL.squeeze().T

imgR = plt.imread('calib_jpg_u/frame_C1_01.jpg')
ret, cornersR = cv2.findChessboardCorners(imgR, (8,6), None)
pts2R = cornersR.squeeze().T

# generate the known 3D point coordinates of points on the checkerboard in cm
pts3 = np.zeros((3,6*8))
yy,xx = np.meshgrid(np.arange(8),np.arange(6))
pts3[0,:] = 2.8*xx.reshape(1,-1)
pts3[1,:] = 2.8*yy.reshape(1,-1)

camL = calibratePose(pts3, pts2L, camL, np.array([0,0,0,1,1,-1]))
camR = calibratePose(pts3, pts2R, camR, np.array([0,0,0,0,0,-1]))

In [15]:
print (camL)
print (camR)

Camera : 
 f=1404.6009650962417 
 c=[[962.16736421 590.91595766]] 
 R=[[ 0.03843674  0.98947412  0.13951197]
 [ 0.9773577  -0.00815434 -0.21143658]
 [-0.20807339  0.14448003 -0.96738358]] 
 t = [[ 6.86588616 19.52347083 47.34419138]]
Camera : 
 f=1404.6009650962417 
 c=[[962.16736421 590.91595766]] 
 R=[[-0.00259871  0.99096865  0.13406858]
 [ 0.99277875 -0.01352251  0.11919517]
 [ 0.11993162  0.1334102  -0.98377748]] 
 t = [[ 7.50010466  7.20926449 47.76495322]]
